In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import metrics
import datetime
from sklearn.ensemble import ExtraTreesRegressor


In [2]:
#function to load the datasets
#C:\\Pri doc\\priti\\Projects\\Myproject\\Berkely\\class project3\\sample project\\data
def prodata():
    train = pd.read_csv('C:\\Pri doc\\priti\\Projects\\Myproject\\Berkely\\class project3\\sample project\\data\\train.csv')
    feature = pd.read_csv('C:\\Pri doc\\priti\\Projects\\Myproject\\Berkely\\class project3\\sample project\\data\\features.csv')
    test = pd.read_csv('C:\\Pri doc\\priti\\Projects\\Myproject\\Berkely\\class project3\\sample project\\data\\test.csv')
    feature = del_unemployment(feature)
    train = del_train_markdown(train)
    return (train,test,feature)


In [3]:
#divide the markdowns into different groups
def del_lack_markdown(feature):
    a = pd.notnull(feature.MarkDown1)
    b = pd.notnull(feature.MarkDown2)
    c = pd.notnull(feature.MarkDown3)
    d = pd.notnull(feature.MarkDown4)
    e = pd.notnull(feature.MarkDown5)
    train = feature[a|b|c|d|e]
    feature = feature[feature.Date >= '2011-11-04']
    return feature


In [4]:
#function to store the features column values
def del_unemployment(feature):
    feature = feature[['Store','Date','Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','IsHoliday']]
    return feature

#function to store required date data
def del_train_markdown(train):
    train = train[train.Date >= '2011-11-11']
    return train


In [5]:
#logic to handle holiday wrt sales
def combi_train_feature(train,test,feature,markdown):
    train = np.array(train)
    test = np.array(test)
    feature = np.array(feature)
    train_x,train_y,test_x,dates=[],[],[],[]
    j=0
    for i in range(len(train)):
        train_x.append([])
        store,dept,date,sales,isholiday = train[i]
        f = find_from_feature(store,date,feature,markdown)
        train_y.append(sales)
        train_x[j] =list(f)
        temp = date.split('-')
        y,m,d =int(temp[0]),int(temp[1]),int(temp[2])
        ymd = datetime.date(y,m,d)
        week = datetime.timedelta(days=7)
        preweek = ymd-week
        preweek = str(preweek)
        pre2week = ymd-week-week
        pre2week = str(pre2week)
        nextweek = ymd+week
        nextweek = str(nextweek)
        next2week = ymd+week+week
        next2week = str(next2week)
        preweek = get_holiday_feature(preweek)
        pre2week = get_holiday_feature(pre2week)
        thisweek = get_holiday_feature(date)
        nextweek = get_holiday_feature(nextweek)
        next2week = get_holiday_feature(next2week)
        train_x[j] =train_x[j]+preweek+thisweek+nextweek+pre2week+next2week
        j += 1
    j = 0
    for i in range(len(test)):
        test_x.append([])
        store,dept,date,isholiday = test[i]
        f = find_from_feature(store,date,feature,markdown)
        test_x[j] = list(f)
        temp = date.split('-')
        y,m,d = int(temp[0]),int(temp[1]),int(temp[2])
        ymd = datetime.date(y,m,d)
        week = datetime.timedelta(days=7)
        preweek = ymd-week
        preweek = str(preweek)
        nextweek = ymd+week
        nextweek = str(nextweek)
        preweek = get_holiday_feature(preweek)
        thisweek = get_holiday_feature(date)
        nextweek = get_holiday_feature(nextweek)
        pre2week = ymd-week-week
        pre2week = str(pre2week)
        next2week = ymd+week+week
        next2week = str(next2week)
        pre2week = get_holiday_feature(pre2week)
        next2week = get_holiday_feature(next2week)
        test_x[j] =test_x[j]+ preweek+thisweek+nextweek+pre2week+next2week
        dates.append(date)
        j += 1
    return (train_x,train_y,test_x,dates)


In [6]:
#logic to handle holiday wrt sales
def find_from_feature(store,date,feature,markdown):
    for i in range(len(feature)):
        if feature[i][0] == store and feature[i][1] == date:
            for j in range(4,9):
                if pd.isnull(feature[i][j]):
                    feature[i][j] = markdown[j-4]
            return feature[i][2:-1]

In [7]:
#model the datasets
def linear_model(train_x,train_y,test_x):
    clf = LinearRegression()
    clf.fit(train_x,train_y)
    test_y = clf.predict(test_x)
    return test_y

In [8]:
def knn_model(train_x,train_y,test_x,k):
    clf = ExtraTreesRegressor(n_estimators=200,max_features='log2')
    clf.fit(train_x,train_y)
    test_y = clf.predict(test_x)
    return test_y

In [9]:
#handle missing values
def nan_rep(trains):
    md = []
    md.append(list(trains.MarkDown1))
    md.append(list(trains.MarkDown2))
    md.append(list(trains.MarkDown3))
    md.append(list(trains.MarkDown4))
    md.append(list(trains.MarkDown5))
    result = []
    for m in md:
        temp = np.array([i for i in m if pd.notnull(i)])
        result.append(temp.mean())
    return result

In [10]:
#handle holidays
def get_holiday_feature(date):
    super_bowl = ['2010-02-12','2011-02-11','2012-02-10','2013-02-08']
    labor = ['2010-09-10','2011-09-09','2012-09-07','2013-09-06']
    thx = ['2010-11-26','2011-11-25','2012-11-23','2013-11-29']
    chris = ['2010-12-31','2011-12-30','2012-12-28','2013-12-27']
    if date in super_bowl:
        return [0,0,0,1]
    elif date in labor:
        return [0,0,1,0]
    elif date in thx:
        return [0,1,0,0]
    elif date in chris:
        return [1,0,0,0]
    else:
        return [0,0,0,0]

In [11]:
#save the output
def write(y,store,dept,dates):
    f = open('result.csv','a')
    for i in range(len(y)):
        Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
        sales = y[i]
        f.write('%s,%s\n'%(Id,sales))
    f.close()
if __name__=="__main__":
    f = open('result.csv','w')
    f.write('Id,Weekly_Sales\n')
    f.close()
    train,test,feature = prodata()
    for i in range(1,46):
        traindata = train[train.Store == i]
        testdata = test[test.Store == i]
        featuredata = feature[feature.Store == i]
        dept_train = list(set(traindata.Dept.values))
        dept_test = list(set(testdata.Dept.values))
        for dept in dept_test:
            if dept not in dept_train:
                print(i,dept)
                tests = testdata[testdata.Dept == dept]
                dates = list(tests.Date)
                y=[0 for j in range(len(tests))]
                write(y,i,dept,dates)
        
        for dept in dept_train:
            trains = traindata[traindata.Dept == dept]
            tests = testdata[testdata.Dept == dept]
            markdown = nan_rep(featuredata)
            print('store=',i,' and dept ',dept)
            train_x,train_y,test_x,dates = combi_train_feature(trains,tests,featuredata,markdown)
            k = 3
            if len(test_x) > 0:
                if len(train_x) <k:
                    test_y = knn_model(train_x,train_y,test_x,len(train_x))
                    write(test_y,i,dept,dates)
                else:
                    test_y = knn_model(train_x,train_y,test_x,k)
                    write(test_y,i,dept,dates)

store= 1  and dept  1
store= 1  and dept  2
store= 1  and dept  3
store= 1  and dept  4
store= 1  and dept  5
store= 1  and dept  6
store= 1  and dept  7
store= 1  and dept  8
store= 1  and dept  9
store= 1  and dept  10
store= 1  and dept  11
store= 1  and dept  12
store= 1  and dept  13
store= 1  and dept  14
store= 1  and dept  16
store= 1  and dept  17
store= 1  and dept  18
store= 1  and dept  19
store= 1  and dept  20
store= 1  and dept  21
store= 1  and dept  22
store= 1  and dept  23
store= 1  and dept  24
store= 1  and dept  25
store= 1  and dept  26
store= 1  and dept  27
store= 1  and dept  28
store= 1  and dept  29
store= 1  and dept  30
store= 1  and dept  31
store= 1  and dept  32
store= 1  and dept  33
store= 1  and dept  34
store= 1  and dept  35
store= 1  and dept  36
store= 1  and dept  37
store= 1  and dept  38
store= 1  and dept  40
store= 1  and dept  41
store= 1  and dept  42
store= 1  and dept  44
store= 1  and dept  45
store= 1  and dept  46
store= 1  and dept  

store= 5  and dept  90
store= 5  and dept  91
store= 5  and dept  92
store= 5  and dept  94
store= 5  and dept  95
store= 5  and dept  96
store= 5  and dept  97
store= 5  and dept  98
store= 6  and dept  1
store= 6  and dept  2
store= 6  and dept  3
store= 6  and dept  4
store= 6  and dept  5
store= 6  and dept  6
store= 6  and dept  7
store= 6  and dept  8
store= 6  and dept  9
store= 6  and dept  10
store= 6  and dept  11
store= 6  and dept  12
store= 6  and dept  13
store= 6  and dept  14
store= 6  and dept  16
store= 6  and dept  17
store= 6  and dept  18
store= 6  and dept  19
store= 6  and dept  20
store= 6  and dept  21
store= 6  and dept  22
store= 6  and dept  23
store= 6  and dept  24
store= 6  and dept  25
store= 6  and dept  26
store= 6  and dept  27
store= 6  and dept  28
store= 6  and dept  29
store= 6  and dept  30
store= 6  and dept  31
store= 6  and dept  32
store= 6  and dept  33
store= 6  and dept  34
store= 6  and dept  35
store= 6  and dept  36
store= 6  and dept  

store= 10  and dept  59
store= 10  and dept  60
store= 10  and dept  67
store= 10  and dept  71
store= 10  and dept  72
store= 10  and dept  74
store= 10  and dept  77
store= 10  and dept  79
store= 10  and dept  80
store= 10  and dept  81
store= 10  and dept  82
store= 10  and dept  83
store= 10  and dept  85
store= 10  and dept  87
store= 10  and dept  90
store= 10  and dept  91
store= 10  and dept  92
store= 10  and dept  93
store= 10  and dept  94
store= 10  and dept  95
store= 10  and dept  96
store= 10  and dept  97
store= 10  and dept  98
store= 11  and dept  1
store= 11  and dept  2
store= 11  and dept  3
store= 11  and dept  4
store= 11  and dept  5
store= 11  and dept  6
store= 11  and dept  7
store= 11  and dept  8
store= 11  and dept  9
store= 11  and dept  10
store= 11  and dept  11
store= 11  and dept  12
store= 11  and dept  13
store= 11  and dept  14
store= 11  and dept  16
store= 11  and dept  17
store= 11  and dept  18
store= 11  and dept  19
store= 11  and dept  20
s

store= 15  and dept  19
store= 15  and dept  20
store= 15  and dept  21
store= 15  and dept  22
store= 15  and dept  23
store= 15  and dept  24
store= 15  and dept  25
store= 15  and dept  26
store= 15  and dept  27
store= 15  and dept  28
store= 15  and dept  29
store= 15  and dept  30
store= 15  and dept  31
store= 15  and dept  32
store= 15  and dept  33
store= 15  and dept  34
store= 15  and dept  35
store= 15  and dept  36
store= 15  and dept  37
store= 15  and dept  38
store= 15  and dept  40
store= 15  and dept  41
store= 15  and dept  42
store= 15  and dept  44
store= 15  and dept  45
store= 15  and dept  46
store= 15  and dept  47
store= 15  and dept  49
store= 15  and dept  50
store= 15  and dept  51
store= 15  and dept  52
store= 15  and dept  54
store= 15  and dept  55
store= 15  and dept  56
store= 15  and dept  58
store= 15  and dept  59
store= 15  and dept  60
store= 15  and dept  67
store= 15  and dept  71
store= 15  and dept  72
store= 15  and dept  74
store= 15  and d

store= 19  and dept  74
store= 19  and dept  77
store= 19  and dept  78
store= 19  and dept  79
store= 19  and dept  80
store= 19  and dept  81
store= 19  and dept  82
store= 19  and dept  83
store= 19  and dept  85
store= 19  and dept  87
store= 19  and dept  90
store= 19  and dept  91
store= 19  and dept  92
store= 19  and dept  93
store= 19  and dept  94
store= 19  and dept  95
store= 19  and dept  96
store= 19  and dept  97
store= 19  and dept  98
store= 19  and dept  99
store= 20  and dept  1
store= 20  and dept  2
store= 20  and dept  3
store= 20  and dept  4
store= 20  and dept  5
store= 20  and dept  6
store= 20  and dept  7
store= 20  and dept  8
store= 20  and dept  9
store= 20  and dept  10
store= 20  and dept  11
store= 20  and dept  12
store= 20  and dept  13
store= 20  and dept  14
store= 20  and dept  16
store= 20  and dept  17
store= 20  and dept  18
store= 20  and dept  19
store= 20  and dept  20
store= 20  and dept  21
store= 20  and dept  22
store= 20  and dept  23
s

store= 24  and dept  25
store= 24  and dept  26
store= 24  and dept  27
store= 24  and dept  28
store= 24  and dept  29
store= 24  and dept  30
store= 24  and dept  31
store= 24  and dept  32
store= 24  and dept  33
store= 24  and dept  34
store= 24  and dept  35
store= 24  and dept  36
store= 24  and dept  37
store= 24  and dept  38
store= 24  and dept  40
store= 24  and dept  41
store= 24  and dept  42
store= 24  and dept  44
store= 24  and dept  45
store= 24  and dept  46
store= 24  and dept  47
store= 24  and dept  49
store= 24  and dept  50
store= 24  and dept  52
store= 24  and dept  54
store= 24  and dept  55
store= 24  and dept  56
store= 24  and dept  58
store= 24  and dept  59
store= 24  and dept  60
store= 24  and dept  67
store= 24  and dept  71
store= 24  and dept  72
store= 24  and dept  74
store= 24  and dept  77
store= 24  and dept  79
store= 24  and dept  80
store= 24  and dept  81
store= 24  and dept  82
store= 24  and dept  83
store= 24  and dept  85
store= 24  and d

store= 28  and dept  85
store= 28  and dept  87
store= 28  and dept  90
store= 28  and dept  91
store= 28  and dept  92
store= 28  and dept  93
store= 28  and dept  94
store= 28  and dept  95
store= 28  and dept  96
store= 28  and dept  97
store= 28  and dept  98
store= 28  and dept  99
29 48
store= 29  and dept  1
store= 29  and dept  2
store= 29  and dept  3
store= 29  and dept  4
store= 29  and dept  5
store= 29  and dept  6
store= 29  and dept  7
store= 29  and dept  8
store= 29  and dept  9
store= 29  and dept  10
store= 29  and dept  11
store= 29  and dept  12
store= 29  and dept  13
store= 29  and dept  14
store= 29  and dept  16
store= 29  and dept  17
store= 29  and dept  18
store= 29  and dept  19
store= 29  and dept  20
store= 29  and dept  21
store= 29  and dept  22
store= 29  and dept  23
store= 29  and dept  24
store= 29  and dept  25
store= 29  and dept  26
store= 29  and dept  27
store= 29  and dept  28
store= 29  and dept  29
store= 29  and dept  30
store= 29  and dept

store= 33  and dept  81
store= 33  and dept  82
store= 33  and dept  83
store= 33  and dept  87
store= 33  and dept  90
store= 33  and dept  91
store= 33  and dept  92
store= 33  and dept  93
store= 33  and dept  94
store= 33  and dept  95
store= 33  and dept  96
store= 33  and dept  97
store= 33  and dept  98
store= 33  and dept  99
34 39
store= 34  and dept  1
store= 34  and dept  2
store= 34  and dept  3
store= 34  and dept  4
store= 34  and dept  5
store= 34  and dept  6
store= 34  and dept  7
store= 34  and dept  8
store= 34  and dept  9
store= 34  and dept  10
store= 34  and dept  11
store= 34  and dept  12
store= 34  and dept  13
store= 34  and dept  14
store= 34  and dept  16
store= 34  and dept  17
store= 34  and dept  18
store= 34  and dept  19
store= 34  and dept  20
store= 34  and dept  21
store= 34  and dept  22
store= 34  and dept  23
store= 34  and dept  24
store= 34  and dept  25
store= 34  and dept  26
store= 34  and dept  27
store= 34  and dept  28
store= 34  and dept

store= 38  and dept  97
store= 38  and dept  98
store= 38  and dept  99
store= 39  and dept  1
store= 39  and dept  2
store= 39  and dept  3
store= 39  and dept  4
store= 39  and dept  5
store= 39  and dept  6
store= 39  and dept  7
store= 39  and dept  8
store= 39  and dept  9
store= 39  and dept  10
store= 39  and dept  11
store= 39  and dept  12
store= 39  and dept  13
store= 39  and dept  14
store= 39  and dept  16
store= 39  and dept  17
store= 39  and dept  18
store= 39  and dept  19
store= 39  and dept  20
store= 39  and dept  21
store= 39  and dept  22
store= 39  and dept  23
store= 39  and dept  24
store= 39  and dept  25
store= 39  and dept  26
store= 39  and dept  27
store= 39  and dept  28
store= 39  and dept  29
store= 39  and dept  30
store= 39  and dept  31
store= 39  and dept  32
store= 39  and dept  33
store= 39  and dept  34
store= 39  and dept  35
store= 39  and dept  36
store= 39  and dept  38
store= 39  and dept  40
store= 39  and dept  41
store= 39  and dept  42
s

store= 43  and dept  97
store= 43  and dept  98
store= 43  and dept  99
store= 44  and dept  1
store= 44  and dept  2
store= 44  and dept  3
store= 44  and dept  4
store= 44  and dept  5
store= 44  and dept  6
store= 44  and dept  7
store= 44  and dept  8
store= 44  and dept  9
store= 44  and dept  10
store= 44  and dept  11
store= 44  and dept  12
store= 44  and dept  13
store= 44  and dept  14
store= 44  and dept  16
store= 44  and dept  17
store= 44  and dept  18
store= 44  and dept  20
store= 44  and dept  21
store= 44  and dept  22
store= 44  and dept  23
store= 44  and dept  24
store= 44  and dept  25
store= 44  and dept  26
store= 44  and dept  27
store= 44  and dept  28
store= 44  and dept  31
store= 44  and dept  32
store= 44  and dept  33
store= 44  and dept  38
store= 44  and dept  40
store= 44  and dept  42
store= 44  and dept  44
store= 44  and dept  46
store= 44  and dept  49
store= 44  and dept  52
store= 44  and dept  55
store= 44  and dept  56
store= 44  and dept  59
s